In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from os import makedirs, listdir
from os.path import join, isfile, isdir, exists, splitext
from pprint import pprint
import json
Settings = json.load(open('settings.txt'))
pprint(Settings)
print("")
from pak.datasets.MOT import MOT16
from pak import utils
import sys
sys.path.append('../')

from cabbage.features.deepmatching import DeepMatching
import cabbage.features.spatio as st

# -------------------------

root = Settings['data_root']
mot16 = MOT16(root)

delta_max = 5
dm = DeepMatching(Settings['deepmatch'], join(root, 'deep_matching'),
                 delta_max=delta_max)

VIDEO = "MOT16-11"
img_loc = mot16.get_test_imgfolder(VIDEO)

X, Y_det, Y_gt = mot16.get_train(VIDEO, memmapped=True)

{'data_root': '/Users/basheq/Desktop/RGBD_Lab/Data',
 'deepmatch': '/Users/basheq/Desktop/RGBD_Lab/deepmatching_1.2.2_c++/deepmatching'}

/Users/basheq/Desktop/RGBD_Lab/Data/MOT16 found :)
MOT16: load memmapped file /Users/basheq/Desktop/RGBD_Lab/Data/MOT16/train/MOT16-11/data.memmap
MOT16 X loaded
MOT16 Y_det loaded
MOT16 Y_gt loaded


## Generate Deep Matches

For performance reasons we need to pre-generate the deep matches using the API functions

In [2]:
dm.generate_matches(img_loc, VIDEO)

{DM}: solve 1 -> 1
{DM}: solve 1 -> 2
{DM}: solve 1 -> 3
{DM}: solve 1 -> 4
{DM}: solve 1 -> 5
{DM}: solve 1 -> 6
{DM}: solve 2 -> 2
{DM}: solve 2 -> 3
{DM}: solve 2 -> 4
{DM}: solve 2 -> 5
{DM}: solve 2 -> 6
{DM}: solve 2 -> 7
{DM}: solve 3 -> 3
{DM}: solve 3 -> 4
{DM}: solve 3 -> 5
{DM}: solve 3 -> 6
{DM}: solve 3 -> 7
{DM}: solve 3 -> 8
{DM}: solve 4 -> 4
{DM}: solve 4 -> 5
{DM}: solve 4 -> 6
{DM}: solve 4 -> 7
{DM}: solve 4 -> 8
{DM}: solve 4 -> 9
{DM}: solve 5 -> 5
{DM}: solve 5 -> 6
{DM}: solve 5 -> 7
{DM}: solve 5 -> 8
{DM}: solve 5 -> 9
{DM}: solve 5 -> 10
{DM}: solve 6 -> 6
{DM}: solve 6 -> 7
{DM}: solve 6 -> 8
{DM}: solve 6 -> 9
{DM}: solve 6 -> 10
{DM}: solve 6 -> 11
{DM}: solve 7 -> 7
{DM}: solve 7 -> 8
{DM}: solve 7 -> 9
{DM}: solve 7 -> 10
{DM}: solve 7 -> 11
{DM}: solve 7 -> 12
{DM}: solve 8 -> 8
{DM}: solve 8 -> 9
{DM}: solve 8 -> 10
{DM}: solve 8 -> 11
{DM}: solve 8 -> 12
{DM}: solve 8 -> 13
{DM}: solve 9 -> 9
{DM}: solve 9 -> 10
{DM}: solve 9 -> 11
{DM}: solve 9 -> 12

{DM}: solve 66 -> 71
{DM}: solve 67 -> 67
{DM}: solve 67 -> 68
{DM}: solve 67 -> 69
{DM}: solve 67 -> 70
{DM}: solve 67 -> 71
{DM}: solve 67 -> 72
{DM}: solve 68 -> 68
{DM}: solve 68 -> 69
{DM}: solve 68 -> 70
{DM}: solve 68 -> 71
{DM}: solve 68 -> 72
{DM}: solve 68 -> 73
{DM}: solve 69 -> 69
{DM}: solve 69 -> 70
{DM}: solve 69 -> 71
{DM}: solve 69 -> 72
{DM}: solve 69 -> 73
{DM}: solve 69 -> 74
{DM}: solve 70 -> 70
{DM}: solve 70 -> 71
{DM}: solve 70 -> 72
{DM}: solve 70 -> 73
{DM}: solve 70 -> 74
{DM}: solve 70 -> 75
{DM}: solve 71 -> 71
{DM}: solve 71 -> 72
{DM}: solve 71 -> 73
{DM}: solve 71 -> 74
{DM}: solve 71 -> 75
{DM}: solve 71 -> 76
{DM}: solve 72 -> 72
{DM}: solve 72 -> 73
{DM}: solve 72 -> 74
{DM}: solve 72 -> 75
{DM}: solve 72 -> 76
{DM}: solve 72 -> 77
{DM}: solve 73 -> 73
{DM}: solve 73 -> 74
{DM}: solve 73 -> 75
{DM}: solve 73 -> 76
{DM}: solve 73 -> 77
{DM}: solve 73 -> 78
{DM}: solve 74 -> 74
{DM}: solve 74 -> 75
{DM}: solve 74 -> 76
{DM}: solve 74 -> 77
{DM}: solve 7

{DM}: solve 129 -> 129
{DM}: solve 129 -> 130
{DM}: solve 129 -> 131
{DM}: solve 129 -> 132
{DM}: solve 129 -> 133
{DM}: solve 129 -> 134
{DM}: solve 130 -> 130
{DM}: solve 130 -> 131
{DM}: solve 130 -> 132
{DM}: solve 130 -> 133
{DM}: solve 130 -> 134
{DM}: solve 130 -> 135
{DM}: solve 131 -> 131
{DM}: solve 131 -> 132
{DM}: solve 131 -> 133
{DM}: solve 131 -> 134
{DM}: solve 131 -> 135
{DM}: solve 131 -> 136
{DM}: solve 132 -> 132
{DM}: solve 132 -> 133
{DM}: solve 132 -> 134
{DM}: solve 132 -> 135
{DM}: solve 132 -> 136
{DM}: solve 132 -> 137
{DM}: solve 133 -> 133
{DM}: solve 133 -> 134
{DM}: solve 133 -> 135
{DM}: solve 133 -> 136
{DM}: solve 133 -> 137
{DM}: solve 133 -> 138
{DM}: solve 134 -> 134
{DM}: solve 134 -> 135
{DM}: solve 134 -> 136
{DM}: solve 134 -> 137
{DM}: solve 134 -> 138
{DM}: solve 134 -> 139
{DM}: solve 135 -> 135
{DM}: solve 135 -> 136
{DM}: solve 135 -> 137
{DM}: solve 135 -> 138
{DM}: solve 135 -> 139
{DM}: solve 135 -> 140
{DM}: solve 136 -> 136
{DM}: solve

{DM}: solve 188 -> 191
{DM}: solve 188 -> 192
{DM}: solve 188 -> 193
{DM}: solve 189 -> 189
{DM}: solve 189 -> 190
{DM}: solve 189 -> 191
{DM}: solve 189 -> 192
{DM}: solve 189 -> 193
{DM}: solve 189 -> 194
{DM}: solve 190 -> 190
{DM}: solve 190 -> 191
{DM}: solve 190 -> 192
{DM}: solve 190 -> 193
{DM}: solve 190 -> 194
{DM}: solve 190 -> 195
{DM}: solve 191 -> 191
{DM}: solve 191 -> 192
{DM}: solve 191 -> 193
{DM}: solve 191 -> 194
{DM}: solve 191 -> 195
{DM}: solve 191 -> 196
{DM}: solve 192 -> 192
{DM}: solve 192 -> 193
{DM}: solve 192 -> 194
{DM}: solve 192 -> 195
{DM}: solve 192 -> 196
{DM}: solve 192 -> 197
{DM}: solve 193 -> 193
{DM}: solve 193 -> 194
{DM}: solve 193 -> 195
{DM}: solve 193 -> 196
{DM}: solve 193 -> 197
{DM}: solve 193 -> 198
{DM}: solve 194 -> 194
{DM}: solve 194 -> 195
{DM}: solve 194 -> 196
{DM}: solve 194 -> 197
{DM}: solve 194 -> 198
{DM}: solve 194 -> 199
{DM}: solve 195 -> 195
{DM}: solve 195 -> 196
{DM}: solve 195 -> 197
{DM}: solve 195 -> 198
{DM}: solve

{DM}: solve 248 -> 248
{DM}: solve 248 -> 249
{DM}: solve 248 -> 250
{DM}: solve 248 -> 251
{DM}: solve 248 -> 252
{DM}: solve 248 -> 253
{DM}: solve 249 -> 249
{DM}: solve 249 -> 250
{DM}: solve 249 -> 251
{DM}: solve 249 -> 252
{DM}: solve 249 -> 253
{DM}: solve 249 -> 254
{DM}: solve 250 -> 250
{DM}: solve 250 -> 251
{DM}: solve 250 -> 252
{DM}: solve 250 -> 253
{DM}: solve 250 -> 254
{DM}: solve 250 -> 255
{DM}: solve 251 -> 251
{DM}: solve 251 -> 252
{DM}: solve 251 -> 253
{DM}: solve 251 -> 254
{DM}: solve 251 -> 255
{DM}: solve 251 -> 256
{DM}: solve 252 -> 252
{DM}: solve 252 -> 253
{DM}: solve 252 -> 254
{DM}: solve 252 -> 255
{DM}: solve 252 -> 256
{DM}: solve 252 -> 257
{DM}: solve 253 -> 253
{DM}: solve 253 -> 254
{DM}: solve 253 -> 255
{DM}: solve 253 -> 256
{DM}: solve 253 -> 257
{DM}: solve 253 -> 258
{DM}: solve 254 -> 254
{DM}: solve 254 -> 255
{DM}: solve 254 -> 256
{DM}: solve 254 -> 257
{DM}: solve 254 -> 258
{DM}: solve 254 -> 259
{DM}: solve 255 -> 255
{DM}: solve

{DM}: solve 307 -> 310
{DM}: solve 307 -> 311
{DM}: solve 307 -> 312
{DM}: solve 308 -> 308
{DM}: solve 308 -> 309
{DM}: solve 308 -> 310
{DM}: solve 308 -> 311
{DM}: solve 308 -> 312
{DM}: solve 308 -> 313
{DM}: solve 309 -> 309
{DM}: solve 309 -> 310
{DM}: solve 309 -> 311
{DM}: solve 309 -> 312
{DM}: solve 309 -> 313
{DM}: solve 309 -> 314
{DM}: solve 310 -> 310
{DM}: solve 310 -> 311
{DM}: solve 310 -> 312
{DM}: solve 310 -> 313
{DM}: solve 310 -> 314
{DM}: solve 310 -> 315
{DM}: solve 311 -> 311
{DM}: solve 311 -> 312
{DM}: solve 311 -> 313
{DM}: solve 311 -> 314
{DM}: solve 311 -> 315
{DM}: solve 311 -> 316
{DM}: solve 312 -> 312
{DM}: solve 312 -> 313
{DM}: solve 312 -> 314
{DM}: solve 312 -> 315
{DM}: solve 312 -> 316
{DM}: solve 312 -> 317
{DM}: solve 313 -> 313
{DM}: solve 313 -> 314
{DM}: solve 313 -> 315
{DM}: solve 313 -> 316
{DM}: solve 313 -> 317
{DM}: solve 313 -> 318
{DM}: solve 314 -> 314
{DM}: solve 314 -> 315
{DM}: solve 314 -> 316
{DM}: solve 314 -> 317
{DM}: solve

{DM}: solve 367 -> 367
{DM}: solve 367 -> 368
{DM}: solve 367 -> 369
{DM}: solve 367 -> 370
{DM}: solve 367 -> 371
{DM}: solve 367 -> 372
{DM}: solve 368 -> 368
{DM}: solve 368 -> 369
{DM}: solve 368 -> 370
{DM}: solve 368 -> 371
{DM}: solve 368 -> 372
{DM}: solve 368 -> 373
{DM}: solve 369 -> 369
{DM}: solve 369 -> 370
{DM}: solve 369 -> 371
{DM}: solve 369 -> 372
{DM}: solve 369 -> 373
{DM}: solve 369 -> 374
{DM}: solve 370 -> 370
{DM}: solve 370 -> 371
{DM}: solve 370 -> 372
{DM}: solve 370 -> 373
{DM}: solve 370 -> 374
{DM}: solve 370 -> 375
{DM}: solve 371 -> 371
{DM}: solve 371 -> 372
{DM}: solve 371 -> 373
{DM}: solve 371 -> 374
{DM}: solve 371 -> 375
{DM}: solve 371 -> 376
{DM}: solve 372 -> 372
{DM}: solve 372 -> 373
{DM}: solve 372 -> 374
{DM}: solve 372 -> 375
{DM}: solve 372 -> 376
{DM}: solve 372 -> 377
{DM}: solve 373 -> 373
{DM}: solve 373 -> 374
{DM}: solve 373 -> 375
{DM}: solve 373 -> 376
{DM}: solve 373 -> 377
{DM}: solve 373 -> 378
{DM}: solve 374 -> 374
{DM}: solve

{DM}: solve 426 -> 429
{DM}: solve 426 -> 430
{DM}: solve 426 -> 431
{DM}: solve 427 -> 427
{DM}: solve 427 -> 428
{DM}: solve 427 -> 429
{DM}: solve 427 -> 430
{DM}: solve 427 -> 431
{DM}: solve 427 -> 432
{DM}: solve 428 -> 428
{DM}: solve 428 -> 429
{DM}: solve 428 -> 430
{DM}: solve 428 -> 431
{DM}: solve 428 -> 432
{DM}: solve 428 -> 433
{DM}: solve 429 -> 429
{DM}: solve 429 -> 430
{DM}: solve 429 -> 431
{DM}: solve 429 -> 432
{DM}: solve 429 -> 433
{DM}: solve 429 -> 434
{DM}: solve 430 -> 430
{DM}: solve 430 -> 431
{DM}: solve 430 -> 432
{DM}: solve 430 -> 433
{DM}: solve 430 -> 434
{DM}: solve 430 -> 435
{DM}: solve 431 -> 431
{DM}: solve 431 -> 432
{DM}: solve 431 -> 433
{DM}: solve 431 -> 434
{DM}: solve 431 -> 435
{DM}: solve 431 -> 436
{DM}: solve 432 -> 432
{DM}: solve 432 -> 433
{DM}: solve 432 -> 434
{DM}: solve 432 -> 435
{DM}: solve 432 -> 436
{DM}: solve 432 -> 437
{DM}: solve 433 -> 433
{DM}: solve 433 -> 434
{DM}: solve 433 -> 435
{DM}: solve 433 -> 436
{DM}: solve

KeyboardInterrupt: 

## Get a Match

In [ ]:
M = dm.get_match(VIDEO, frame1=1, frame2=2)
print(M.shape)

In [ ]:
frame1, _, x, y, w, h, _, _, _ = Y_gt[0]


fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(121)
ax.imshow(X[0])
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='red')
bb1 = (x,y,w,h)

frame2, _, x, y, w, h, _, _, _ = Y_gt[2]
ax = fig.add_subplot(122)
ax.imshow(X[1])
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='red')
bb2 = (x,y,w,h)


frame3, _, x, y, w, h, _, _, _ = Y_gt[601]
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='blue')
bb3 = (x,y,w,h)
# ----------------------

cost1 = dm.calculate_cost(VIDEO, frame1, bb1, frame2, bb2)
cost2 = dm.calculate_cost(VIDEO, frame1, bb1, frame3, bb3)
print("dm cost (red vs red):\t", cost1)
print("dm cost (red vs blue):\t", cost2)

cost1 = st.calculate(bb1, bb2)
cost2 = st.calculate(bb1, bb3)
print("st cost (red vs red):\t", cost1)
print("st cost (red vs blue):\t", cost2)
print(X.shape)

plt.show()